<a href="https://colab.research.google.com/github/MdNiamul/DataHioding/blob/main/Hidden14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#!/usr/bin/env python3
"""
Hybrid Auto + Partial Recovery LSB Decoder
- Auto trial-error decoding (length-header + sentinel)
- If fail, fallback to partial extraction
- Supports Hamming(7,4), all bit/channel/lsb/endian combinations
"""

import zlib
from PIL import Image
from google.colab import files
from typing import List

# --- Bit utils ---
def bits_to_int(bits: List[int], endian="big") -> int:
    val=0
    if endian=="big":
        for b in bits: val=(val<<1)|b
    else:
        for i,b in enumerate(bits): val|=(b<<i)
    return val

def bits_to_bytes(bits: List[int]) -> bytes:
    if len(bits)%8!=0: bits=bits[:len(bits)-(len(bits)%8)]
    out=bytearray()
    for i in range(0,len(bits),8):
        byte=0
        for b in bits[i:i+8]: byte=(byte<<1)|b
        out.append(byte)
    return bytes(out)

def bytes_to_bits(data: bytes) -> List[int]:
    bits=[]
    for b in data:
        for i in range(7,-1,-1): bits.append((b>>i)&1)
    return bits

# --- Hamming(7,4) ---
def _hamming74_syndrome(code: List[int]) -> int:
    p1=code[0]^code[2]^code[4]^code[6]
    p2=code[1]^code[2]^code[5]^code[6]
    p3=code[3]^code[4]^code[5]^code[6]
    return (p1<<2)|(p2<<1)|p3

def hamming74_decode(bits: List[int]) -> List[int]:
    out=[]
    for i in range(0,len(bits)-len(bits)%7,7):
        block=bits[i:i+7]
        syn=_hamming74_syndrome(block)
        if syn!=0:
            pos=syn-1
            if 0<=pos<7: block[pos]^=1
        out.extend([block[2],block[4],block[5],block[6]])
    return out

# --- LSB extract ---
def extract_lsb_bits(img: Image.Image, bits_per_channel=1, channels="RGB", lsb_order="lsb") -> List[int]:
    channels=channels.upper()
    allowed={"R":0,"G":1,"B":2}
    idxs=[allowed[c] for c in channels if c in allowed]
    if img.mode!="RGB": img=img.convert("RGB")
    w,h=img.size; px=img.load(); out_bits=[]
    for y in range(h):
        for x in range(w):
            r,g,b=px[x,y]; vals=(r,g,b)
            for ch in idxs:
                v=vals[ch]
                rng=range(bits_per_channel) if lsb_order=="lsb" else reversed(range(bits_per_channel))
                for k in rng: out_bits.append((v>>k)&1)
    return out_bits

# --- Decoders ---
def decode_length_header(bitstream: List[int], ecc="none", endian="big") -> bytes:
    if len(bitstream)<64: raise ValueError("Too short for length header")
    length=bits_to_int(bitstream[:32], endian)
    crc_expected=bits_to_int(bitstream[32:64], endian)
    payload_bits=bitstream[64:]
    if ecc=="hamming": payload_bits=hamming74_decode(payload_bits)
    needed=length*8
    if len(payload_bits)<needed: raise ValueError("Not enough bits")
    data=bits_to_bytes(payload_bits[:needed])
    if zlib.crc32(data)&0xFFFFFFFF != crc_expected: raise ValueError("CRC mismatch")
    return data

def decode_sentinel(bitstream: List[int], sentinel=b"EOF!", ecc="none") -> bytes:
    if ecc=="hamming": bitstream=hamming74_decode(bitstream)
    sent_bits=bytes_to_bits(sentinel)
    for i in range(len(bitstream)-len(sent_bits)+1):
        if bitstream[i:i+len(sent_bits)]==sent_bits:
            return bits_to_bytes(bitstream[:i])
    raise ValueError("Sentinel not found")

# --- Hybrid Auto + Partial Decoder ---
def run_hybrid_decoder():
    print("📤 Upload PNG/BMP image:")
    uploaded=files.upload()
    if not uploaded: return
    img_path=list(uploaded.keys())[0]
    img=Image.open(img_path)

    bits_opts=[1,2,3]
    channels_opts=["R","G","B","RG","RB","GB","RGB"]
    modes=["length","sentinel"]
    ecc_opts=["none","hamming"]
    endian_opts=["big","little"]
    lsb_order_opts=["lsb","msb"]
    sentinel=b"EOF!"

    print("🔍 Starting hybrid trial-error decoding ...")
    for bits in bits_opts:
        for channels in channels_opts:
            for mode in modes:
                for ecc in ecc_opts:
                    for endian in endian_opts:
                        for lsb_order in lsb_order_opts:
                            try:
                                bits_stream=extract_lsb_bits(img,bits,channels,lsb_order)
                                if mode=="length": data=decode_length_header(bits_stream,ecc=ecc,endian=endian)
                                else: data=decode_sentinel(bits_stream,sentinel=sentinel,ecc=ecc)
                                out_file=f"decoded_{mode}_{bits}bits_{channels}_{ecc}_{endian}_{lsb_order}.bin"
                                with open(out_file,"wb") as f: f.write(data)
                                print(f"[✅] Success! Extracted {len(data)} bytes -> {out_file}")
                                files.download(out_file)
                                return
                            except: continue

    # If all auto decode fails, fallback to partial extraction
    print("⚠️ Auto decode failed, starting partial recovery ...")
    for bits in bits_opts:
        for channels in channels_opts:
            for lsb_order in lsb_order_opts:
                try:
                    bits_stream=extract_lsb_bits(img,bits,channels,lsb_order)
                    data=bits_to_bytes(bits_stream)
                    out_file=f"partial_{bits}bits_{channels}_{lsb_order}.bin"
                    with open(out_file,"wb") as f: f.write(data)
                    print(f"[✅] Partial extracted {len(data)} bytes -> {out_file}")
                    files.download(out_file)
                except Exception as e:
                    print(f"[❌] Failed {bits}bits {channels} {lsb_order}: {e}")

# Run
run_hybrid_decoder()


📤 Upload PNG/BMP image:


Saving bengali_stego_encoded_20250819_105945.png to bengali_stego_encoded_20250819_105945 (3).png
🔍 Starting hybrid trial-error decoding ...
⚠️ Auto decode failed, starting partial recovery ...
[✅] Partial extracted 4995 bytes -> partial_1bits_R_lsb.bin


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[✅] Partial extracted 4995 bytes -> partial_1bits_R_msb.bin


...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[✅] Partial extracted 44955 bytes -> partial_3bits_RGB_msb.bin


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>